# Simple Gen AI App using Langchain

# # Load Data --> Docs  -->Divide our text into chunks --> Vectors --> Vector Embeddings --> Vector Store DB

In [ ]:
#Import all The Libraries and API Keys
import os 
from dotenv import load_dotenv

from langchain_community.llms import Ollama
import streamlit 

load_dotenv()





# load the data

In [5]:
#Data Ingestion--> From the website we need to scrap the data

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui")
final_load = loader.load()

In [6]:
final_load

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt engineering quickstart - Docs by LangChain', 'language': 'en'}, page_content="Prompt engineering quickstart - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationPrompt engineering quickstartGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewQuickstartConceptsCreate and update promptsCreate a promptManage promptsManage prompts programmaticallyConfigure prompt settingsUse tools in a promptInclude multimodal content in a promptWrite your prompt with AIConnect to modelsTutorialsOptimize a classifierSync prompts with GitHubTest multi-turn conversationsOn this pagePrerequisitesNext stepsVideo guidePrompt engineering quickstartCopy pageCopy pagePrompts gui

# Divides the data into Chunks

In [7]:
# Load Data --> Docs  -->Divide our text into chunks --> Vectors --> Vector Embeddings --> Vector Store DB

from langchain_text_splitters import RecursiveCharacterTextSplitter

web_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)

final_split = web_splitter.split_documents(final_load)

In [8]:
final_split

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt engineering quickstart - Docs by LangChain', 'language': 'en'}, page_content="Prompt engineering quickstart - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationPrompt engineering quickstartGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewQuickstartConceptsCreate and update promptsCreate a promptManage promptsManage prompts programmaticallyConfigure prompt settingsUse tools in a promptInclude multimodal content in a promptWrite your prompt with AIConnect to modelsTutorialsOptimize a classifierSync prompts with GitHubTest multi-turn conversationsOn this pagePrerequisitesNext stepsVideo guidePrompt engineering quickstartCopy pageCopy pagePrompts gui

# Converting Small Chunks into Vector

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="gemma:2b")
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='gemma:2b', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

# Setup Faiss 

In [11]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(final_split,embeddings)
db

# Query Perform

In [12]:
#Query From a Vector DB
query = " navigate to the Prompts section of the left-hand sidebar and click on "
result = db.similarity_search(query)
result

[Document(id='4a240d51-7030-4e21-bb82-f0ed7de2314a', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt engineering quickstart - Docs by LangChain', 'language': 'en'}, page_content='Under the Prompts heading select the gear  icon next to the model name, which will launch the Prompt Settings window on the Model Configuration tab.\n\n\nSet the model configuration you want to use. The Provider and Model you select will determine the parameters that are configurable on this configuration page. Once set, click Save as.\n\n\n\nSpecify the input variables you would like to test in the Inputs box and then click  Start.\n\nTo learn about more options for configuring your prompt in the Playground, refer to Configure prompt settings.\n\n\nAfter testing and refining your prompt, click Save to store it for future use.\n\n\u200b4. Iterate on a promptLangSmith allows for team-based prompt iteration. Workspace members can experiment wi

In [13]:
result[0].page_content

'Under the Prompts heading select the gear  icon next to the model name, which will launch the Prompt Settings window on the Model Configuration tab.\n\n\nSet the model configuration you want to use. The Provider and Model you select will determine the parameters that are configurable on this configuration page. Once set, click Save as.\n\n\n\nSpecify the input variables you would like to test in the Inputs box and then click  Start.\n\nTo learn about more options for configuring your prompt in the Playground, refer to Configure prompt settings.\n\n\nAfter testing and refining your prompt, click Save to store it for future use.\n\n\u200b4. Iterate on a promptLangSmith allows for team-based prompt iteration. Workspace members can experiment with prompts in the playground and save their changes as a new commit when ready.To improve your prompts:\n\nReference the documentation provided by your model provider for best practices in prompt creation, such as:'

In [14]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = "llama-3.1-8b-instant")
print(llm)


profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True} client=<groq.resources.chat.completions.Completions object at 0x000002D396094B00> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D3B5B82BA0> model_name='llama-3.1-8b-instant' model_kwargs={} groq_api_key=SecretStr('**********')


# Retrieval Chain

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.documents import Document

# Initialize Groq LLM
llm = ChatGroq(model="llama-3.1-8b-instant")

# Prompt template
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context:

<context>
{context}
</context>

Question: {input}
""")

# Compose runnable chain
document_chain = prompt | llm

# Example documents
docs = [
    Document(page_content="LangChain is a framework for building applications with LLMs."),
    Document(page_content="Groq provides fast inference for LLMs like Llama 3."),
]

# Convert documents into a single string
context_text = "\n".join([doc.page_content for doc in docs])

# Run the chain
question = "What is LangChain and how does Groq help?"

result = document_chain.invoke({
    "context": context_text,
    "input": question
})

print(result)


content="Based on the provided context, here's the answer to your question:\n\nLangChain is a framework for building applications with Large Language Models (LLMs). It provides a way to integrate LLMs into various applications and workflows, leveraging their capabilities to generate text, answer questions, and perform other language-related tasks.\n\nGroq helps by providing fast inference for LLMs, such as Llama 3, which means it accelerates the processing of LLMs' responses. This can be particularly useful for applications that require high-performance and low-latency language processing, such as chatbots, virtual assistants, and real-time language translation systems. By using Groq's fast inference capabilities, developers can build more efficient and scalable applications that can handle large volumes of language-based data and tasks." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 91, 'total_tokens': 249, 'completion_time': 0.2512

In [20]:
from langchain_core.documents import Document


In [23]:
#Input --> Retrievers --> VectorStoreDB
db

In [28]:
retriever = db.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)


ModuleNotFoundError: No module named 'langchain.chains'

In [ ]:
# Get the response from 
retrieval_chain.invoke("LangChain is a framework for building applications with LLMs")
response["answer"]
response[̣'context']

SyntaxError: invalid character '̣' (U+0323) (882800435.py, line 4)